In [ ]:
# 安装必要的库
%pip install rapidocr_onnxruntime -i https://pypi.tuna.tsinghua.edu.cn/simple
%pip install "unstructured[all-docs]" -i https://pypi.tuna.tsinghua.edu.cn/simple
%pip install pyMuPDF -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
from langchain.document_loaders import PyMuPDFLoader

# 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径
loader = PyMuPDFLoader("/workspaces/LLMEasyProject/database/knowledge_db/pumpkin_book.pdf")

# 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
pages = loader.load()

In [ ]:
print(f"载入后的变量类型为：{type(pages)}，",  f"该 PDF 一共包含 {len(pages)} 页")

In [ ]:
page = pages[1]
print(f"每一个元素的类型：{type(page)}.", 
    f"该文档的描述性数据：{page.metadata}", 
    f"查看该文档的内容:\n{page.page_content[0:100]}", 
    sep="\n------\n")


In [ ]:
# 文档分割
from langchain.text_splitter import RecursiveCharacterTextSplitter
# 知识库中单段文本长度
CHUNK_SIZE = 500

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

from langchain.document_loaders import PyMuPDFLoader

# 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径
loader = PyMuPDFLoader("/workspaces/LLMEasyProject/database/knowledge_db/pumpkin_book.pdf")

# 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
pages = loader.load()
page = pages[1]

# 使用递归字符文本分割器
from langchain.text_splitter import TokenTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
text_splitter.split_text(page.page_content[0:1000])

In [ ]:
split_docs = text_splitter.split_documents(pages)
print(f"切分后的文件数量：{len(split_docs)}")


In [ ]:
print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")


In [ ]:
%pip install sentence-transformers
# 文档词向量化
# 采用huggingface的免费模型
import os
import openai
import zhipuai
import sys

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings


embedding = HuggingFaceEmbeddings(model_name="moka-ai/m3e-base")

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

query1 = "机器学习"
query2 = "强化学习"
query3 = "大语言模型"

# 通过对应的 embedding 类生成 query 的 embedding。
emb1 = embedding.embed_query(query1)
emb2 = embedding.embed_query(query2)
emb3 = embedding.embed_query(query3)

# 将返回结果转成 numpy 的格式，便于后续计算
emb1 = np.array(emb1)
emb2 = np.array(emb2)
emb3 = np.array(emb3)


print(f"{query1} 生成的为长度 {len(emb1)} 的 embedding , 其前 30 个值为： {emb1[:30]}") 


In [ ]:
print(f"{query1} 和 {query2} 向量之间的点积为：{np.dot(emb1, emb2)}")
print(f"{query1} 和 {query3} 向量之间的点积为：{np.dot(emb1, emb3)}")
print(f"{query2} 和 {query3} 向量之间的点积为：{np.dot(emb2, emb3)}")


In [ ]:
print(f"{query1} 和 {query2} 向量之间的余弦相似度为：{cosine_similarity(emb1.reshape(1, -1) , emb2.reshape(1, -1) )}")
print(f"{query1} 和 {query3} 向量之间的余弦相似度为：{cosine_similarity(emb1.reshape(1, -1) , emb3.reshape(1, -1) )}")
print(f"{query2} 和 {query3} 向量之间的余弦相似度为：{cosine_similarity(emb2.reshape(1, -1) , emb3.reshape(1, -1) )}")


In [1]:
# 向量数据库
# Langchain 集成了超过 30 个不同的向量存储库。我们选择 Chroma 是因为它轻量级且数据存储在内存中，这使得它非常容易启动和开始使用。

# %pip instll pydantic-settings==2.0.1
# %pip install chromadb
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings


from langchain.llms import OpenAI
from langchain.llms import HuggingFacePipeline

import os
import openai
import zhipuai
import sys

loaders_chinese = [
    PyMuPDFLoader("/workspaces/LLMEasyProject/database/knowledge_db/pumpkin_book.pdf") # 南瓜书
    # 大家可以自行加入其他文件
]

docs = []

for loader in loaders_chinese:
    docs.extend(loader.load())
# 切分文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)
split_docs = text_splitter.split_documents(docs)

embedding = HuggingFaceEmbeddings(model_name="moka-ai/m3e-base")

persist_directory = '/workspaces/LLMEasyProject/database/vector_db/chroma'

vectordb = Chroma.from_documents(
    documents=split_docs[:100], # 为了速度，只选择了前 100 个切分的 doc 进行生成。
    embedding=embedding,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)
vectordb.persist()


vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)
print(f"向量库中存储的数量：{vectordb._collection.count()}")


向量库中存储的数量：400


In [2]:
question="什么是机器学习"
sim_docs = vectordb.similarity_search(question,k=5)
print(f"检索到的内容数：{len(sim_docs)}")

for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")


检索到的内容数：5
检索到的第0个内容: 
→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解 第 2 版》
←_←
第 7 章 贝叶斯分类器
62
7.1
贝叶斯决策论 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
62
7.1.1
式 (7.5) 的推导 . . . . . . . . . . . . 
--------------
检索到的第1个内容: 
→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解 第 2 版》
←_←
第 7 章 贝叶斯分类器
62
7.1
贝叶斯决策论 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
62
7.1.1
式 (7.5) 的推导 . . . . . . . . . . . . 
--------------
检索到的第2个内容: 
→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解 第 2 版》
←_←
第 7 章 贝叶斯分类器
62
7.1
贝叶斯决策论 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
62
7.1.1
式 (7.5) 的推导 . . . . . . . . . . . . 
--------------
检索到的第3个内容: 
→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解 第 2 版》
←_←
9.2.4
式 (9.8) 的解释 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
99
9.2.5
式 (9.12) 的解释
. . . . . . . . . . . . . . . . . . . . . 
--------------
检索到的第4个内容: 
→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解 第 2 版》
←_←
9.2.4
式 (9.8) 的解释 . . . 

In [ ]:
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)
for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")


In [2]:
# 导入检索式问答链
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
# %conda install sentencepiece
from langchain import PipelineAI
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer
from transformers import AutoModel
# llm = OpenAI(temperature=0)
# 可以使用 HuggingFacePipeline 本地搭建大语言模型
model_id = 'THUDM/chatglm2-6b-int4' # 采用 int 量化后的模型可以节省硬盘占用以及实时量化所需的运算资源
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id, trust_remote_code=True).half().quantize(4).cuda()
model = model.eval()
pipe = PipelineAI(
    "text2text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=100
)

llm = HuggingFacePipeline(pipeline=pipe)

# 声明一个检索式问答链
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)
# 可以以该方式进行检索问答
question = "本知识库主要包含什么内容"
result = qa_chain({"query": question})
print(f"大语言模型的回答为：{result['result']}")


: 